# Create AML environment

Consider having these resources created through a CI/CD pipeline

## Packages

In [1]:
from azureml.core import Workspace
from azureml.core import Environment as Environment1
from azure.ai.ml.entities import Environment as Environment2
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import os

## Upload DataBook wheel

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='paolt-ml-v2', subscription_id='fadb1e32-9c96-4180-be9d-1811f4687cca', resource_group='paolt-ml-v2')

It is necessary to build the *wheel* for the *DataBook* package. To do so, run the following from the command prompt:

```
cd databook_module
python setup.py bdist_wheel
```

Then we upload the whel file to a BLOB and, once uploaded, the wheel file have to be specified in the conda yml file

In [3]:
whl_path = "../databook_module/dist/databook-0.1-py3-none-any.whl"
whl_url = Environment1.add_private_pip_wheel(workspace=ws, file_path=whl_path, exist_ok=True)

In [4]:
print(f"Add {whl_url} to ./conda.yml as a conda dependency")

Add https://paoltmlv28403383412.blob.core.windows.net/azureml/Environment/azureml-private-packages/databook-0.1-py3-none-any.whl to ./conda.yml


## Create an environment


We are creating an AML Custom Environment based on a curated one (Sklearn-CPU)

In [5]:
ml_client = MLClient.from_config(
    credential=DefaultAzureCredential())
ml_client

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/paolt-dev-cpu-vm/code/Users/paolt/databook_v1/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f833e2ee430>,
         subscription_id=fadb1e32-9c96-4180-be9d-1811f4687cca,
         resource_group_name=paolt-ml-v2,
         workspace_name=paolt-ml-v2)

In [6]:
databook_env = Environment2(
    name="DataBook-Env",
    description="Custom environment for DataBook v-1.0",
    tags={"DataBook ver": "v-1.0"},
    conda_file=os.path.join('.', "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    version="1.0",
)

In [ ]:
# databook_env.python.conda_dependencies.add_pip_package(whl_url)


We now register the environment to AML as a custom environment, and we build the reative image

In [ ]:
# databook_env.register(ws)

In [8]:
databook_env = ml_client.environments.create_or_update(databook_env)

In [10]:
print(
    f"Environment with name {databook_env.name} is registered to workspace, the environment version is {databook_env.version}"
)

Environment with name DataBook-Env is registered to workspace, the environment version is 1.0
